In [ ]:
Object Detection
Panorama stiching
Augmented Reality
Image Search

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img_train = cv2.imread('LabSession6Images/box_in_scene.png',0) # trainImage
img_query = cv2.imread('LabSession6Images/box.png',0) # queryImage

sift = cv2.xfeatures2d.SIFT_create()
kp_query = sift.detect(img_query)
kp_train = sift.detect(img_train)
kp_query, des_query = sift.compute(img_query, kp_query)
kp_train, des_train = sift.compute(img_train, kp_train)
 
    
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)
# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

(matches, H, status) = M
result = cv2.warpPerspective(imageA, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
 
# check to see if the keypoint matches should be visualized
if showMatches:
vis = self.drawMatches(imageA, imageB, kpsA, kpsB, matches,status)

